In [1]:
from santiago_core import SANTIAGO_CORE
import pandas as pd
pd.set_option('display.max_columns', 500)
import constants as const
from datetime import timedelta, date, datetime

In [2]:
sa = SANTIAGO_CORE()